In [7]:
# from google.colab import drive
# drive.mount('/content/drive/')
# !ls '/content/drive/MyDrive/07MIAR_PracticaObservacional/code/'

# # para añadir un directorio de trabajo y poder importar las clases y funciones personalizadads
# import sys
# sys.path.append('/content/drive/MyDrive/07MIAR_PracticaObservacional/code/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
convolutional_autoencoder.py  __pycache__	  utils.py
my_data_generator.py	      training_CAE.ipynb


In [1]:
# Imports necesarios
import tensorflow as tf
import pandas as pd
from my_data_generator import DataGenerator
from convolutional_autoencoder import ConvAutoencoder
from utils import save_reconstructed_images, create_environment, create_json
from tensorflow.keras.models import load_model
import os
import numpy as np
import cv2

ModuleNotFoundError: No module named 'skimage'

In [11]:
#Hyper-parameters
input_dim = (128, 128, 1)                     # las dimensiones de entrada
encoder_conv_filters = [32, 64, 64, 64]       # el numero de mapas de activacion en cada bloque convolucional
encoder_conv_kernel_size = [3, 3, 3, 3]       # las dimensiones de los kernels de cada bloque convolucional
encoder_conv_strides = [2, 2, 2, 2]           # los strides de cada bloque conv
decoder_conv_t_filters = [64, 64, 32, 1]      # filtros de cada bloque convolucional de decoder, son inversos a los anteriores
decoder_conv_t_kernel_size = [3, 3, 3, 3]     # tamaño de los kernels
decoder_conv_t_strides = [2, 2, 2, 2]         # los strides del decoer

# habre una sesion de tensorflow
sess = tf.function()                          
z_dim = 200             # dimesniones del espacio latente
lr = 0.0005             # el learning_rate
batch_size = 8          # tamaño de lotes de entrenamiento
epochs = 50             # número de epocas
r_loss_factor = 0.4     # factor de reconstruccion
is_training = True

In [15]:
base_folder = "/content/drive/MyDrive/07MIAR_PracticaObservacional/"
#I/O paths
# mapea las carpetas donde va a ir guardando cositas
run_folders = {"tsv_path":      base_folder + "data/partition.csv", 
               "data_path":     base_folder + "data/images/", 
               "model_path":    base_folder + "data/Models/", 
               "results_path":  base_folder + "data/Results/", 
               "log_filename":  base_folder + "data/Results/log/CAE.log"
               }

In [16]:
# Creando el entramado de directorios
create_environment(run_folders)

In [17]:
# pasa los parametros en un diccionario y luego los pasa a json
hyperparameters = {"input_dim":                  input_dim, 
                   "encoder_conv_filters":       encoder_conv_filters, 
                   "encoder_conv_kernel_size":   encoder_conv_kernel_size, 
                   "encoder_conv_strides":       encoder_conv_strides, 
                   "decoder_conv_t_filters":     decoder_conv_t_filters, 
                   "decoder_conv_t_kernel_size": decoder_conv_t_kernel_size, 
                   "decoder_conv_t_strides":     decoder_conv_t_strides, 
                   "z_dim":                      z_dim, 
                   "lr":                         lr, 
                   "batch_size":                 batch_size, 
                   "epochs":                     epochs, 
                   "r_loss_factor":              r_loss_factor, 
                   "opt":                        "Adam", 
                   "loss_function":              "mse",
                   "data_path":                  run_folders["data_path"]
                   }
create_json(hyperparameters, run_folders)

In [18]:
# Fase de entrenamiento
if is_training:

  # Carga de datos
  df_pneumo_2d = pd.read_csv(run_folders["tsv_path"], sep=";")             # crea el dataframe, con separador de puntos y comas
  df_pneumo_2d.columns = ['ImageID', 'Partition', 'Class']                 # asigna columnas como header
  data_filter = df_pneumo_2d['Partition']=='train'                         # separa los datos de train

  # Cargador de datos training
  data_flow_train = DataGenerator(df_pneumo_2d[data_filter],               # se pasa el dataframe filtrado
                                  input_dim[1],                            # ancho
                                  input_dim[0],                            # alto 
                                  input_dim[2],                            # canales
                                  batch_size=batch_size,                   # tamaño del lote
                                  path_to_img=run_folders["data_path"],    # el path de la data
                                  shuffle = True)                          # mezlar: si
  
  # Cargador de datos validation
  data_filter = df_pneumo_2d['Partition']=='val'                           # datos de validación
  data_flow_val = DataGenerator(df_pneumo_2d[data_filter],                 # el dataframe
                                  input_dim[1], 
                                  input_dim[0], 
                                  input_dim[2], 
                                  batch_size=batch_size, 
                                  path_to_img=run_folders["data_path"],
                                  )                                        # esta vez no se mezcla por que shuffle solo interesa en train
  
  # Creando el CAE
  my_CAE = ConvAutoencoder(input_dim,                         # se pasan los dimensiones
                           encoder_conv_filters,              # filtros de ENCODER
                           encoder_conv_kernel_size,          # tamaño del kernel en el ENCODER
                           encoder_conv_strides,              # los pasos del ENCODER
                           decoder_conv_t_filters,            # fitlros de DECODER
                           decoder_conv_t_kernel_size,        # kernels de DECODER
                           decoder_conv_t_strides,            # el paso de DECODER
                           z_dim)                             # dimensiones del espacio latente
  
  # Construyendo arquitectura
  my_CAE.build(use_batch_norm=True,         # utiliza batch normalization
              use_dropout=False,            # no usa dropout
              VCAE=False)                   # utiliza la version original osea un AUTO ENCODER NORMAL
  
  print(my_CAE.model.summary())             # MUESTRA EL RESULMEN

  # Compilando el CAE
  my_CAE.compile(learning_rate=lr, r_loss_factor=r_loss_factor)     

  # Entrenando el CAE
  steps_per_epoch = len(data_flow_train)
  H = my_CAE.train(data_flow_train, 
                   epochs, 
                   steps_per_epoch, 
                   data_flow_val, 
                   run_folders)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 1)]     0         
_________________________________________________________________
encoder_conv0 (Conv2D)       (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 32)        0         
_________________________________________________________________
encoder_conv1 (Conv2D)       (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
44/44 [==============================] - 398s 9s/step - loss: 0.0441 - val_loss: 0.0518

Epoch 00001: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 2/50
44/44 [==============================] - 19s 423ms/step - loss: 0.0194 - val_loss: 0.0391

Epoch 00002: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 3/50
44/44 [==============================] - 19s 422ms/step - loss: 0.0140 - val_loss: 0.0395

Epoch 00003: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 4/50
44/44 [==============================] - 19s 424ms/step - loss: 0.0118 - val_loss: 0.0374

Epoch 00004: saving model to /content/drive/MyDrive/07MIAR_PracticaObservacional/data/Models/exp_0001/weights/CAE_weights.h5
Epoch 5/50
44/44 [==============================] - 19s 423ms/step - loss: 0.0095 - val_loss: 

In [20]:
is_training = True
# Carga de datos
df_pneumo_2d = pd.read_csv(run_folders["tsv_path"], sep=";")
df_pneumo_2d.columns = ['ImageID', 'Partition', 'Class']
data_filter = df_pneumo_2d['Partition']=='test'
# Cargador de datos training
data_flow_test = DataGenerator(df_pneumo_2d[data_filter], 
                                input_dim[1], 
                                input_dim[0], 
                                input_dim[2], 
                                batch_size=batch_size, 
                                path_to_img=run_folders["data_path"],
                                shuffle = False)

if not is_training:
  my_CAE = ConvAutoencoder.load_model(run_folders)          # cargar el modelo     

print(my_CAE.model.summary())

example_batch = data_flow_test.__getitem__(index=0)
example_images = example_batch[0]

y_pred = my_CAE.model.predict(example_images)
print(y_pred.shape)

save_reconstructed_images(example_images, y_pred, run_folders)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 128, 128, 1)]     0         
_________________________________________________________________
encoder_conv0 (Conv2D)       (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 32)        0         
_________________________________________________________________
encoder_conv1 (Conv2D)       (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0   